In [53]:
#Ignore warnings which can clutter

In [54]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Declare Packages

In [56]:
import os #where env variable placed
import litellm
import json  # Import the json library
from crewai import Agent, Crew, Task
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from pydantic import BaseModel

In [57]:
#Connect to Gemini Flash &Seper with api key

In [ ]:
gemini_api_key = os.getenv("GEMINI_API_KEY")
my_llm = "gemini/gemini-2.0-flash"
serper_api_key = os.getenv("SERPER_API_KEY")

#if gemini_api_key is None:
#    raise ValueError("API key environment variable not set.")
#print(f"API Key: {gemini_api_key}")


#if serper_api_key is None:
#    raise ValueError("Serper_API key environment variable not set.")
#print(f"API Key: {serper_api_key}")

In [59]:
#Set Input Parameters Dynamic(5)
#-->Crew(4)
    #-->Task(3)
      ##-->Agent(2)
            #-->Tool(1)

In [60]:
#CrewAI Tools (Initialize the tools(1))

In [61]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

MY FIRST AGENTS (RAM, LAKSHMAN & BHARATAN)

In [ ]:
# Agent 1: Venue Coordinator
ram_venue_coordinator_agnt = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    allow_delegation=False,
    llm=my_llm,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
            )
)

# Agent 3: Marketing and Communications Agent
lakshman_marketing_communications_agnt = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=my_llm,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
        )
)

    # Agent 3: Marketing and Communications Agent
bharatan_flight_finder_agnt = Agent(
    role="Flight Coordinator",
    goal="Identify and book an appropriate flight "
    "based on travel requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    allow_delegation=False,
    llm=my_llm,
    backstory=(
        "With a keen sense of space and "
        "understanding of travel needs, "
        "you excel at finding and securing "
        "the budgetted flight which achieve travel needs, "
        "cost and budget constraints."
        )
)

    

In [64]:
## Creating Venue Pydantic Object
##- Agents will populate this object with information about different venues by creating different instances of it.

In [65]:
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int

class FlightDetails(BaseModel):
    flight_name: str
    flight_cost: int
    flight_name: str
    travel_time: int
    #booking_status: str

In [66]:
##Creating Task (3)

In [67]:
find_venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    #human_input=False,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=ram_venue_coordinator_agnt
)

find_flight_task = Task(
    description="Find a flight to travel from {travel_from} "
                "Find a flight to travel to {event_city} "
                "Find a flight to travel on {travel_date} "
                "that meets criteria for return per passenger is  {travel_budget}",    
    expected_output="All the details of a specifically chosen"
                    "flight you found to accommodate the travel.",
    #human_input=False,
    async_execution=False,
    output_json=FlightDetails,
    output_file="flight_details.json",  
      # Outputs the venue details as a JSON file
    agent=bharatan_flight_finder_agnt
)

marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=lakshman_marketing_communications_agnt
)

In [68]:
##Creating Single Crew which has task->agent->tool (3)
##samelike a job, we need a team_prjt-->task-->person-->his skillset to execute

In [69]:
event_management_crew = Crew(
    agents=[ram_venue_coordinator_agnt, 
            #,logistics_manager,  -->on leave
            lakshman_marketing_communications_agnt,
            bharatan_flight_finder_agnt
           ], 
    tasks=[find_venue_task,
           find_flight_task,
          # ,logistics_task, 
           marketing_task
          ],
    verbose=True
)

2025-02-26 11:34:41,693 - 15968 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [70]:
#set input parameters (DYNAMIC-Can be updated)

In [71]:
event_details_requirements = {
    'event_topic': "AI tech Conference",
    'event_description': "A gathering of Data and leadership stakeholders "
                         "and industry leaders "
                         "to explore AI & future technologies.",
    'event_city': "Kuala Lumpur",
    'travel_from' : "Manila",
    'travel_date' : "2025-03-25",
    'travel_budget' : "RM 600",
    'tentative_date': "2025-03-26",
    'expected_participants': 80,
    'budget': 7000,
    'venue_type': "Conference Hall"
}

In [72]:
#Project kickoff

In [73]:
result = event_management_crew.kickoff(inputs=event_details_requirements)

# Agent: Venue Coordinator
## Task: Find a venue in Kuala Lumpur that meets criteria for AI tech Conference.


# Agent: Venue Coordinator
## Thought: Okay, I understand. I need to find a venue in Kuala Lumpur for an AI tech conference and provide the venue's name, address, and capacity in a specific JSON format.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"AI tech conference venues Kuala Lumpur\"}"
## Tool Output: 
{'searchParameters': {'q': 'AI tech conference venues Kuala Lumpur', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Tech conference events in Kuala Lumpur, Malaysia - Eventbrite', 'link': 'https://www.eventbrite.com/d/malaysia--kuala-lumpur/tech-conference/', 'snippet': '2025 EdTec – Int Conf on Edu & Learning Tech, 05-06 May, Kuala Lumpur Mon, May 5 • 9:00 AM Hotel Capitol Kuala Lumpur From $165.64', 'position': 1}, {'title': 'Artificial intelligence events in Kuala Lumpur, Malaysia - Eventbrite', 'link': 'ht